In [1]:
import pandas as pd
from geopy.geocoders import Nominatim

In [2]:
def clean_again(df):
    assert df['lat'].isnull().sum() == df['lng'].isnull().sum(), 'input: lat(%d) and lng(%d) must be equal' % (df['lat'].isnull().sum(), df['lng'].isnull().sum())
    to_clean = df[df['turst']==False].copy()
    print 'Remain:', (to_clean['turst'] == False).sum()
    tursted = df[df['turst']==True].copy()
    for idx in to_clean.index:
        geolocator = Nominatim()
        addr = to_clean.loc[idx,'addr'] + ', Detroit'
        try:
            loc = geolocator.geocode(addr) 
            if loc is None:
                print 'Cannot parse %s' % addr
            else:
                lat, lng = float(loc.latitude), float(loc.longitude)
                if lat is not None and lng is not None:
                    to_clean.loc[idx,'lat'] = lat
                    to_clean.loc[idx,'lng'] = lng
            to_clean.loc[idx,'turst'] = True
        except Exception as e:
            print 'Exception: %s, maybe service time out.' % e
            break
    print 'Remain:', (to_clean['turst'] == False).sum()
    res = pd.concat([tursted, to_clean]).sort_index()
    assert res['lat'].isnull().sum() == res['lng'].isnull().sum(), 'output: lat(%d) and lng(%d) must be equal' % (res['lat'].isnull().sum(), res['lng'].isnull().sum())
    return res

In [3]:
clean_permit = pd.read_csv('clean/permit.csv',index_col=0)

In [29]:
clean_permit = clean_again(clean_permit)

Remain: 192
Exception: Service timed out, maybe service time out.
Remain: 192


In [25]:
clean_permit.to_csv('clean/permit.csv')

In [26]:
clean_violation = pd.read_csv('clean/violation.csv',index_col=0)

In [30]:
clean_violation = clean_again(clean_violation)

Remain: 20853
Exception: Service timed out, maybe service time out.
Remain: 20853


In [28]:
clean_violation.to_csv('clean/violation.csv')